In [2]:
from pymatgen.matproj.rest import MPRester
import os
from pymatgen.core.periodic_table import Element
import json

from pandas import DataFrame
import pandas as pd
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)

In [3]:
with MPRester(os.environ['MAPI_KEY']) as m:
    results = []
    result = m.query(criteria={"band_gap": {"$lt": 1.0, "$gt": 0.3}}, properties=["pretty_formula", 
                                                                                         "spacegroup.symbol",
                                                                                         "nsites",
                                                                                         "icsd_id",
                                                                                         "e_above_hull",
                                                                                         "band_gap",
                                                                                         "task_id",
                                                                                        ])
    if len(result)!=0:
            for i in result:
                results.append(i)

In [4]:
len(results)

6670

In [5]:
with open('1.json', 'w') as outfile:
    json.dump(results, outfile)

In [ ]:
Fail_log = open ('Fail.log','w')
for material in results:
    mpid = material['task_id']
    try:
        v_index, c_index, v_dispersion, c_dispersion = band_info(mpid)
        material['v_index'] = v_index
        material['c_index'] = c_index
        material['v_dispersion'] = v_dispersion
        material['c_dispersion'] = c_dispersion
        if v_dispersion>3 or c_dispersion>3:
            print(mpid, material['pretty_formula'], v_dispersion, c_dispersion)
    except:
        Fail_log.write(mpid+' '+material['pretty_formula']+'\n')

mp-7188 Al2Os 1.5669999999999993 3.6638
mp-559676 SnS 2.6645999999999996 3.2641999999999998
mp-558481 ZnGaNO 1.9394999999999998 4.7539
mp-9383 SrHfN2 1.5669 3.216199999999999
mp-775528 Na2MnO2 0.36450000000000005 3.3697
mp-758070 LiCu2PO4 0.2054999999999998 3.7802
mp-571386 CuCl 1.5813999999999995 4.3759
mp-676561 Ag3SI 1.0854000000000001 4.033899999999999
mp-644497 BaTiO3 1.0311 3.1375
mp-2612 GeTe 3.4960000000000004 3.332700000000001
mp-550437 GdTe2ClO5 0.6909000000000001 3.2845000000000004
mp-4366 Na2PtC2 1.4592999999999998 3.6562
mp-3163 BaSnO3 0.9683999999999999 5.890000000000001
mp-9146 CdHgO2 1.1443 3.5592999999999995


In [ ]:
with open('2.json', 'w') as outfile:
    json.dump(results, outfile)

In [11]:
def band_info(mpid):
    bs = m.get_bandstructure_by_material_id(mpid)
    #print(bs.get_band_gap())
    #print(bs.get_vbm())
    #print(bs.get_cbm())
    v_index = list(bs.get_vbm()['band_index'].values())[-1][-1]
    c_index = list(bs.get_cbm()['band_index'].values())[-1][0]
    bands_v = bs.as_dict()['bands']['1'][v_index]
    bands_c = bs.as_dict()['bands']['1'][c_index]
    v_dispersion = max(bands_v)-min(bands_v)
    c_dispersion = max(bands_c)-min(bands_c)
    #print(v_index, max(bands_v)-min(bands_v))
    #print(c_index, max(bands_c)-min(bands_c))
    return v_index, c_index, v_dispersion, c_dispersion



In [27]:
len(bs.as_dict()['kpoints'])

164

AttributeError: 'list' object has no attribute 'values'